<a href="https://colab.research.google.com/github/Jzorro23/Tecs_Internet/blob/main/Descriptores_Geom%C3%A9tricos_NotasProfe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!wget https://www.dropbox.com/s/sn6299q48lywx0x/LetrasABXYZ.zip
!unzip -qq LetrasABXYZ.zip

--2025-10-29 00:29:18--  https://www.dropbox.com/s/sn6299q48lywx0x/LetrasABXYZ.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/xah9i3ds4vsnv2kc0kgjh/LetrasABXYZ.zip?rlkey=cb6ob34agyssqf15sein66c2l [following]
--2025-10-29 00:29:18--  https://www.dropbox.com/scl/fi/xah9i3ds4vsnv2kc0kgjh/LetrasABXYZ.zip?rlkey=cb6ob34agyssqf15sein66c2l
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘LetrasABXYZ.zip.3’

LetrasABXYZ.zip.3       [ <=>                ] 179.62K  --.-KB/s    in 0.06s   

2025-10-29 00:29:18 (2.81 MB/s) - ‘LetrasABXYZ.zip.3’ saved [183932]

[LetrasABXYZ.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import skimage
import cv2

from   tqdm.auto import tqdm

from   scipy.ndimage import binary_fill_holes
from   skimage.segmentation import find_boundaries

In [3]:
# Convierte un número en cadena con ceros a la izquierda
def conv_num_str(x, d):
    # '%0*d' formatea el número x con un ancho total d y rellena con ceros a la izquierda.
    st = '%0*d' % (d, x)
    return st  # Devuelve el número como string con el formato especificado

# Carga una imagen construyendo automáticamente su nombre de archivo
def cargar_imagen(prefijo, num_clase, digitos_clase, num_img, digitos_img, sep='_', verbose='off'):
    # Construye el nombre del archivo de la imagen:
    # ejemplo: prefijo='img', num_clase=3, num_img=7 → "img03_007.png"
    st = (prefijo
          + conv_num_str(num_clase, digitos_clase)  # Convierte el número de clase con ceros a la izquierda
          + sep                                    # Agrega el separador (por defecto '_')
          + conv_num_str(num_img, digitos_img)     # Convierte el número de imagen con ceros a la izquierda
          + '.png')                                # Extensión del archivo

    # Si verbose está activado, imprime el nombre del archivo que se abrirá
    if verbose == 'on':
        print('abriendo imagen ' + st + '...')

    # Lee la imagen usando matplotlib y la devuelve como un arreglo NumPy
    img = plt.imread(st)
    return img  # Devuelve la imagen como un arreglo NumPy

In [4]:
def conv_num_str(x,d):
    st = '%0*d' % (d,x)
    return st

def cargar_imagen(prefijo,num_clase,digitos_clase,num_img,digitos_img,sep='_',verbose='off'):
  st   = prefijo + conv_num_str(num_clase,digitos_clase) + sep + conv_num_str(num_img,digitos_img) + '.png'
  if verbose == 'on':
    print('abriendo image '+st+'...')
  img    = plt.imread(st)
  return img

In [5]:
# Función para calcular descriptores de Fourier de una región binaria R
def des_fourier(R, n_des=16, names=False):
    # Rellenar agujeros en la región binaria R y convertirla a tipo uint8
    R8h = binary_fill_holes(R).astype(np.uint8)

    # Encontrar los contornos en la imagen binaria
    contorno, jerarquia = cv2.findContours(R8h, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Convertir los contornos encontrados en un array de numpy
    x = np.array(contorno)

    # Ajustar la forma del array para obtener las coordenadas del contorno
    B = x.reshape(x.shape[1], 2)

    # Convertir las coordenadas del contorno a números complejos
    V = B[:, 0] + 1j * B[:, 1]
    m = B.shape[0]  # Número de puntos en el contorno

    # Inicialización de arrays para almacenar distancias y ángulos
    r = np.zeros(m, dtype=complex)  # Distancias complejas entre puntos del contorno
    phi = np.zeros(m)  # Ángulos
    dphi = np.zeros(m)  # Diferencias de ángulos
    l = np.zeros(m)  # Longitudes acumuladas a lo largo del contorno
    dl = np.zeros(m)  # Longitudes entre puntos consecutivos

    # Calcular la distancia entre el primer y último punto del contorno
    r[0] = V[0] - V[m-1]

    # Calcular distancias entre puntos consecutivos del contorno
    r[1:] = V[1:] - V[:m-1]

    # Calcular la longitud entre puntos consecutivos
    dl = np.abs(r)

    # Calcular los ángulos de los vectores r
    phi = np.angle(r)

    # Calcular las diferencias de ángulos entre puntos consecutivos
    dphi[:m-1] = np.mod(phi[1:] - phi[:m-1] + np.pi, 2 * np.pi) - np.pi
    dphi[m-1] = np.mod(phi[0] - phi[m-1] + np.pi, 2 * np.pi) - np.pi

    # Calcular la longitud acumulada a lo largo del contorno
    l[0] = dl[0]
    for k in range(1, m):
        l[k] = l[k-1] + dl[k]

    # Normalizar las longitudes acumuladas a una escala de [0, 2π]
    l = l * (2 * np.pi / l[m-1])

    # Inicializar el array de descriptores de Fourier
    descriptores = np.zeros(n_des)

    # Calcular los descriptores de Fourier
    for n in range(1, n_des + 1):
        an = (dphi * np.sin(l * n)).sum()  # Coeficiente 'a' del n-ésimo término
        bn = (dphi * np.cos(l * n)).sum()  # Coeficiente 'b' del n-ésimo término
        an = -an / n / np.pi  # Normalización de 'a'
        bn = bn / n / np.pi   # Normalización de 'b'
        imagi = an + 1j * bn  # Número complejo que combina 'a' y 'b'
        descriptores[n-1] = np.abs(imagi)  # Guardar la magnitud del descriptor

    X = descriptores  # Los descriptores finales son las magnitudes de los términos complejos

    # Si names=True, devolver también los nombres de los descriptores
    if names:
        return np.array([f'Fourier_des {n+1:>2d}' for n in range(n_des)]), descriptores

    return X  # Devolver solo los descriptores si names=False


In [6]:
def momentos_hu(R, nombres=False):
  # regionprops
  # https://scikit-image.org/docs/stable/api/skimage.measure.html#regionprops
  propiedades = skimage.measure.regionprops(R)[0]

  momentos = propiedades.moments_hu

  if nombres:
    nombres_momentos = ['Momento-Hu-1', 'Momento-Hu-2', 'Momento-Hu-3', 'Momento-Hu-4', 'Momento-Hu-5', 'Momento-Hu-6', 'Momento-Hu-7']
    return momentos, nombres_momentos
  else:
    return momentos


In [7]:
def momentos_flusser(R, nombres=False):
  # Calcular los momentos de la región binaria R
  momentos = cv2.moments(R, True)

  # Asignar los momentos centrales necesarios
  u00, u20, u11, u02, u30, u21, u12, u03 = momentos['m00'], momentos['mu20'], momentos['mu11'], momentos['mu02'],
  momentos['mu30'], momentos['mu21'], momentos['mu12'], momentos['mu03']

  # Cálculo de los invariantes de Flusser
  I1 = (u20*u02 - u11**2) / u00**4
  I2 = (u30**2 * u03**2 - 6*u30*u21*u12*u03 + 4*u30*u12**3 + 4*u21**3 * u03 - 3*u21**2 * u12**2) / u00**10
  I3 = (u20*(u21*u03 - u12**2) - u11*(u30*u03 - u21*u12) + u02*(u30*u12 - u21**2)) / u00**7
  I4 = (u20**3*u03**2 - 6*u20**2*u11*u12*u03 - 6*u20**2*u02*u21*u03 + 9*u20**2*u02*u12**2 +
        12*u20*u11**2*u21*u03 + 6*u20*u11*u02*u30*u03 - 18*u20*u11*u02*u21*u12 - 8*u11**3*u30*u03 -
        6*u20*u02**2*u30*u12 + 9*u20*u02**2*u21 + 12*u11**2*u02*u30*u12 - 6*u11*u02**2*u30*u21 + u02**3*u30**2) / u00**11

  # Lista de invariantes calculados
  invariantes = [I1, I2, I3, I4]

  # Si se solicitan los nombres de los invariantes, devolver los nombres también
  if nombres:
    nombres_invariantes = ['Flusser-1', 'Flusser-2', 'Flusser-3', 'Flusser-4']
    return invariantes, nombres_invariantes
  else:
    return invariantes


In [8]:
def momentos_gupta(R, nombres=False):
    # Encontrar los bordes de la región binaria R
    B = skimage.segmentation.find_boundaries(R, mode='inner')

    # Obtener las coordenadas (i_perim, j_perim) de los píxeles que forman el perímetro
    i_perim, j_perim = np.where(B.astype(bool))

    # Representar las coordenadas del perímetro como números complejos
    im_perim = i_perim + j_perim * 1j

    # Calcular el centroide del perímetro
    ix = i_perim.mean()
    jx = j_perim.mean()
    centro = ix + jx * 1j

    # Calcular la distancia entre cada punto del perímetro y el centroide
    z = np.abs(im_perim - centro)
    m1 = z.mean()  # Media de las distancias (primer momento)

    # Desviaciones del primer momento
    mur1 = z - m1
    mur2 = mur1 * mur1  # Segundo momento centrado
    mur3 = mur1 * mur2  # Tercer momento centrado
    mur4 = mur2 * mur2  # Cuarto momento centrado

    # Cálculo de los momentos estadísticos
    mu2 = mur2.mean()  # Segundo momento
    mu3 = mur3.mean()  # Tercer momento
    mu4 = mur4.mean()  # Cuarto momento

    # Cálculo de los descriptores de Gupta
    F1 = (mu2 ** 0.5) / m1      # Primer descriptor de Gupta
    F2 = mu3 / (mu2 * (mu2 ** 0.5))  # Segundo descriptor de Gupta
    F3 = mu4 / mu2 ** 2         # Tercer descriptor de Gupta

    # Crear un array con los tres descriptores
    descriptores = np.array([F1, F2, F3])

    # Si se solicitan los nombres de los descriptores, devolver también los nombres
    if nombres:
        nombres_descriptores = ['Gupta-1', 'Gupta-2', 'Gupta-3']
        return descriptores, nombres_descriptores
    else:
        return descriptores


In [10]:
def geo_basica(R, nombres=False):
    # Centro de masa
    coordenadas = np.argwhere(R)
    coordenadas_fila = coordenadas[:, 0]  # Coordenadas de la fila (i)
    coordenadas_columna = coordenadas[:, 1]  # Coordenadas de la columna (j)
    centro_fila = np.mean(coordenadas_fila)  # Promedio de las filas (centro de masa en el eje i)
    centro_columna = np.mean(coordenadas_columna)  # Promedio de las columnas (centro de masa en el eje j)

    # Altura
    altura = np.max(coordenadas_fila) - np.min(coordenadas_fila) + 1

    # Anchura
    anchura = np.max(coordenadas_columna) - np.min(coordenadas_columna) + 1

    # Área
    area = np.sum(R)

    # Perímetro
    # Usar find_boundaries de skimage para encontrar los bordes del objeto en la imagen binaria
    B = skimage.segmentation.find_boundaries(R, mode='inner')
    perimetro = np.sum(B)

    # Propiedades geométricas avanzadas usando regionprops
    propiedades = skimage.measure.regionprops(R)[0]

    # Circularidad (roundness)
    circularidad = 4 * area * np.pi / (perimetro ** 2)

    # Otras propiedades geométricas
    solidez = propiedades.solidity  # Solidez: área sobre área convexa
    numero_euler = propiedades.euler_number  # Número de Euler
    diametro_equivalente = propiedades.equivalent_diameter_area  # Diámetro equivalente basado en el área
    eje_mayor = propiedades.axis_major_length  # Longitud del eje mayor
    eje_menor = propiedades.axis_minor_length  # Longitud del eje menor
    orientacion = propiedades.orientation  # Orientación del objeto
    extension = propiedades.extent  # Extensión: proporción de área sobre el cuadro delimitador
    excentricidad = propiedades.eccentricity  # Excentricidad
    area_convexa = propiedades.area_convex  # Área convexa

    # Lista con todas las características geométricas calculadas
    caracteristicas = [centro_fila, centro_columna, altura, anchura, area, perimetro, circularidad,
                       numero_euler, diametro_equivalente, eje_mayor, eje_menor, orientacion,
                       solidez, extension, excentricidad, area_convexa]

    # Si se solicitan los nombres de las características, devolver también los nombres
    if nombres:
        nombres_caracteristicas = ['centro_fila', 'centro_columna', 'altura', 'anchura', 'area', 'perimetro', 'circularidad',
                                   'numero_euler', 'diametro_equivalente', 'eje_mayor', 'eje_menor',
                                   'orientacion', 'solidez', 'extension', 'excentricidad', 'area_convexa']
        return caracteristicas, nombres_caracteristicas
    else:
        return caracteristicas


In [11]:
K = 5   # número de clases
N = 100 # número de imágenes por clase

# Características geométricas básicas
Mbg  = 16
Xbg = np.zeros((K*N, Mbg))  # Matriz para almacenar características geométricas básicas

# Descriptores de Fourier
Mfd  = 16
Xfd = np.zeros((K*N, Mfd))  # Matriz para almacenar descriptores de Fourier

# Momentos de Hu
Mhu  = 7
Xhu = np.zeros((K*N, Mhu))  # Matriz para almacenar momentos de Hu

# Momentos de Flusser
Mfl  = 4
Xfl = np.zeros((K*N, Mfl))  # Matriz para almacenar momentos de Flusser

# Momentos de Gupta
Mgu  = 3
Xgu = np.zeros((K*N, Mgu))  # Matriz para almacenar momentos de Gupta

y = np.zeros((K*N), 'int')  # Etiquetas de clase o "ground truth"

t = 0  # Inicializa un contador para llevar el índice total de imágenes procesadas

for j in range(K):  # Bucle sobre el número de clases
    for i in tqdm(range(N)):  # Bucle sobre las imágenes dentro de la clase, con barra de progreso
        # Lectura de la imagen
        img = cargar_imagen('ABXYZ/char_', j+1, 2, i+1, 3, verbose='off')

        # Dilatación de la imagen utilizando un disco de radio 3
        img3 = dilation(img, disk(3))  # Unir la imagen usando dilatación con un disco de radio 3

        # Binarización de la imagen: convertir la imagen en valores binarios (0 y 1)
        R = img3 > 0.5

        # Rellenar los agujeros en las regiones binarias y convertirlo a enteros (0 y 1)
        R = binary_fill_holes(R).astype(int)

        # Extracción de características:
        Xfd[t, :] = des_fourier(R)  # Extraer descriptores de Fourier
        Xhu[t, :] = momentos_hu(R)       # Extraer momentos de Hu
        Xfl[t, :] = momentos_flusser(R)     # Extraer momentos de Flusser
        Xgu[t, :] = momentos_gupta(R)       # Extraer momentos de Gupta
        Xbg[t, :] = geo_basica(R)    # Extraer características geométricas básicas

        # Asignar la etiqueta de clase correspondiente a la imagen procesada
        y[t] = j

        # Incrementar el contador de imágenes procesadas
        t = t + 1


  0%|          | 0/100 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: 'ABXYZ/char_01_001.png'

In [ ]:
x,xn = geo_basica(R,nombres='True')

for k in range(16):
  print(k,xn[k])

0 centro_fila
1 centro_columna
2 altura
3 anchura
4 area
5 perimetro
6 circularidad
7 numero_euler
8 diametro_equivalente
9 eje_mayor
10 eje_menor
11 orientacion
12 solidez
13 extension
14 excentricidad
15 area_convexa


In [ ]:
seleccion_geo = [2,3,4,5,6,7,8,9,10,12,13,14,15]  # características invariantes a la rotación y traslación
Xbg2 = Xbg[:,seleccion_geo]  # <- nuevas características geométricas básicas


In [ ]:
K = 5   # número de clases
N = 100 # número de imágenes por clase

# Características geométricas básicas
Mbg  = 16
Xbg = np.zeros((K*N, Mbg))  # Matriz para almacenar características geométricas básicas

# Descriptores de Fourier
Mfd  = 16
Xfd = np.zeros((K*N, Mfd))  # Matriz para almacenar descriptores de Fourier

# Momentos de Hu
Mhu  = 7
Xhu = np.zeros((K*N, Mhu))  # Matriz para almacenar momentos de Hu

# Momentos de Flusser
Mfl  = 4
Xfl = np.zeros((K*N, Mfl))  # Matriz para almacenar momentos de Flusser

# Momentos de Gupta
Mgu  = 3
Xgu = np.zeros((K*N, Mgu))  # Matriz para almacenar momentos de Gupta

y = np.zeros((K*N), 'int')  # Etiquetas de clase o "ground truth"

t = 0  # Inicializa un contador para llevar el índice total de imágenes procesadas

for j in range(K):  # Bucle sobre el número de clases
    for i in tqdm(range(N)):  # Bucle sobre las imágenes dentro de la clase, con barra de progreso
        # Lectura de la imagen
        img = imageload('LetrasABXYZ/char_', j+1, 2, i+1, 3, echo='off')

        # Dilatación de la imagen utilizando un disco de radio 3
        img3 = dilation(img, disk(3))  # Unir la imagen usando dilatación con un disco de radio 3

        # Binarización de la imagen: convertir la imagen en valores binarios (0 y 1)
        R = img3 > 0.5

        # Rellenar los agujeros en las regiones binarias y convertirlo a enteros (0 y 1)
        R = binary_fill_holes(R).astype(int)

        # Extracción de características:
        Xfd[t, :] = fourierdes(R)  # Extraer descriptores de Fourier
        Xhu[t, :] = hugeo(R)       # Extraer momentos de Hu
        Xfl[t, :] = flusser(R)     # Extraer momentos de Flusser
        Xgu[t, :] = gupta(R)       # Extraer momentos de Gupta
        Xbg[t, :] = basicgeo(R)    # Extraer características geométricas básicas

        # Asignar la etiqueta de clase correspondiente a la imagen procesada
        y[t] = j

        # Incrementar el contador de imágenes procesadas
        t = t + 1


In [ ]:
from itertools import combinations
import warnings
import tqdm

# Función de Fisher para la selección de características
def jfisher(caracteristicas, ypred, p=None):
    m = caracteristicas.shape[1]

    norm = ypred.ravel() - ypred.min()
    max_clase = norm.max() + 1

    if p is None:
        p = np.ones(shape=(max_clase, 1)) / max_clase

    # Centroide de todas las muestras
    media_caracteristicas = caracteristicas.mean(0)

    # Covarianza dentro de la clase
    cov_w = np.zeros(shape=(m, m))

    # Covarianza entre clases
    cov_b = np.zeros(shape=(m, m))

    for k in range(max_clase):
        ii = (norm == k)                                   # Índices de la clase k
        caracteristicas_clase = caracteristicas[ii, :]     # Muestras de la clase k
        media_clase = caracteristicas_clase.mean(0)        # Centroide de la clase k
        covarianza_clase = np.cov(caracteristicas_clase, rowvar=False)  # Covarianza de la clase k

        cov_w += p[k] * covarianza_clase                   # Covarianza dentro de la clase

        dif = (media_clase - media_caracteristicas).reshape((m, 1))
        cov_b += p[k] * dif @ dif.T                        # Covarianza entre clases
    try:
        return np.trace(np.linalg.inv(cov_w) @ cov_b)
    except np.linalg.LinAlgError:
        return -np.inf

In [ ]:
# Implementación de la Selección Secuencial de Características (SFS)
from tqdm import tqdm  # Importa la función tqdm directamente

def sfs_fisher(X, y, p):
    m = X.shape[1]
    caracteristicas_disponibles = np.ones((m,))
    caracteristicas_seleccionadas = -np.ones((p,), int)
    Jmax = 0

    # Bucle para seleccionar p características
    for j in tqdm(range(p)):
        for k in range(m):  # Bucle sobre cada característica disponible
            if caracteristicas_disponibles[k]:  # Si la característica k está disponible
                caracteristicas_seleccionadas[j] = k
                columnas = caracteristicas_seleccionadas[0:j+1]
                Xsel = X[:, columnas]
                J = jfisher(Xsel, y)
                if J > Jmax:
                    Jmax = J
                    ksel = k
                    print('> seleccionada: J(' + str(columnas) + ') = ' + str(J / 100))
        caracteristicas_disponibles[ksel] = 0
        caracteristicas_seleccionadas[j] = ksel

    return caracteristicas_seleccionadas


In [ ]:
# Separación entre entrenamiento y prueba
# Las primeras n muestras de X serán para el entrenamiento, el resto
# para la prueba
def DividirDatos(X, y, n):

    K = np.max(y) + 1              # Número de clases
    N = int(X.shape[0] / K)        # Número de muestras por clase
    Ntrain = n * K                 # Número de muestras para el entrenamiento
    Ntest = K * N - Ntrain         # Número de muestras para la prueba
    M = X.shape[1]                 # Número de características por muestra
    Xtrain = np.zeros((Ntrain, M))  # Subconjunto de entrenamiento
    ytrain = np.zeros((Ntrain), 'int')  # Ground truth (etiquetas) de entrenamiento
    Xtest = np.zeros((Ntest, M))    # Subconjunto de prueba
    ytest = np.zeros((Ntest), 'int')   # Ground truth (etiquetas) de prueba

    # Contadores
    itrain = 0
    itest = 0
    t = 0

    for j in range(K):  # Para cada clase
        for i in range(N):  # Para cada imagen de la clase
            if i < n:  # Si es para entrenamiento
                Xtrain[itrain, :] = X[t, :]
                ytrain[itrain] = y[t]
                itrain = itrain + 1
            else:  # Si es para prueba
                Xtest[itest, :] = X[t, :]
                ytest[itest] = y[t]
                itest = itest + 1
            t = t + 1

    return Xtrain, ytrain, Xtest, ytest

In [ ]:
# Entrada
X_original = np.concatenate((Xbg2, Xfd, Xhu, Xfl, Xgu), axis=1)  # Concatenar todas las características

# Dividir en Entrenamiento/Prueba
Xtrain_original, ytrain, Xtest_original, ytest = DividirDatos(X_original, y, 75)  # Las primeras n muestras de cada clase son entrenamiento, el resto es prueba


In [ ]:
# Función para limpiar las características
def limpiar(caracteristicas, mostrar=False):
    n_caracteristicas = caracteristicas.shape[1]
    ip = np.ones(n_caracteristicas, dtype=int)

    # Limpieza de características correlacionadas
    warnings.filterwarnings('ignore')
    C = np.abs(np.corrcoef(caracteristicas, rowvar=False))
    idxs = np.vstack(np.where(C > .99))

    # Eliminar pares de la misma característica (una característica tendrá correlación de 1 consigo misma)
    idxs = idxs[:, idxs[0, :] != idxs[1, :]]

    # Eliminar características correlacionadas
    if idxs.size > 0:
        ip[np.max(idxs, 0)] = 0

    # Eliminar características constantes
    s = caracteristicas.std(axis=0, ddof=1)
    ip[s < 1e-8] = 0
    p = np.where(ip.astype(bool))[0]

    if mostrar:
        print(f'Limpiar: número de características reducido de {n_caracteristicas} a {p.size}.')

    return p

In [ ]:
# Limpieza de características
sclean = limpiar(Xtrain_original)  # Índices de las características seleccionadas (sin las características constantes)
Xtrain_clean = Xtrain_original[:, sclean]  # Nuevos datos de entrenamiento sin características constantes
Xtest_clean = Xtest_original[:, sclean]    # Nuevos datos de prueba sin características constantes

print(sclean)  # Imprimir los índices de las características seleccionadas
print(sclean.shape)  # Imprimir la forma de los índices seleccionados

[ 0  1  2  3  4  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 39 40 41 42]
(40,)


In [ ]:
def minmax(X):
    ma = X.max(0)      # El valor máximo de cada columna (característica)
    mi = X.min(0)      # El valor mínimo de cada columna (característica)
    md = ma - mi       # Diferencia entre el valor máximo y mínimo (rango)
    a = 1 / md         # El coeficiente de escalado (1/rango)
    b = -mi / md       # El término de traslación (que mueve el mínimo al 0)
    return X * a + b, a, b  # Devuelve los datos normalizados, junto con 'a' y 'b'


In [ ]:
#Selección SFS con Fisher
p  = 15 # número de características a seleccionar
ss = sfs_fisher(Xtrain_norm,ytrain,p)

 13%|█▎        | 2/15 [00:00<00:00, 16.66it/s]

> seleccionada: J([0]) = 4.4453444901265056e-05
> seleccionada: J([1]) = 0.0012298941077666192
> seleccionada: J([2]) = 0.005908993352206653
> seleccionada: J([3]) = 0.006651953300911958
> seleccionada: J([4]) = 0.03811883620597881
> seleccionada: J([12]) = 0.06072655506861238
> seleccionada: J([13]) = 0.14688281034968875
> seleccionada: J([13  0]) = 0.1495930675954814
> seleccionada: J([13  1]) = 0.15376574292072565
> seleccionada: J([13  3]) = 0.1828521033171121
> seleccionada: J([13  4]) = 0.26254982264393695
> seleccionada: J([13 12]) = 0.272646717380343
> seleccionada: J([13 12  0]) = 0.2747012119312428
> seleccionada: J([13 12  1]) = 0.27958686110805425
> seleccionada: J([13 12  2]) = 0.2870586970961179
> seleccionada: J([13 12  3]) = 0.2918373505427225
> seleccionada: J([13 12  4]) = 0.361939576706788
> seleccionada: J([13 12  4  0]) = 0.3648003505425171
> seleccionada: J([13 12  4  1]) = 0.36999566816923235
> seleccionada: J([13 12  4  2]) = 0.38531030319822723
> seleccionada: 

 53%|█████▎    | 8/15 [00:00<00:00, 19.19it/s]

> seleccionada: J([13 12  4 17  0]) = 0.4143052378165176
> seleccionada: J([13 12  4 17  1]) = 0.4204094801872823
> seleccionada: J([13 12  4 17  2]) = 0.433848996201061
> seleccionada: J([13 12  4 17 11]) = 0.4531666922942671
> seleccionada: J([13 12  4 17 14]) = 0.4607546369307034
> seleccionada: J([13 12  4 17 14  0]) = 0.4637694841194624
> seleccionada: J([13 12  4 17 14  1]) = 0.4646814772840401
> seleccionada: J([13 12  4 17 14  2]) = 0.4728309392451393
> seleccionada: J([13 12  4 17 14  5]) = 0.47864382163241226
> seleccionada: J([13 12  4 17 14 11]) = 0.5030824830469117
> seleccionada: J([13 12  4 17 14 11  0]) = 0.505793616897644
> seleccionada: J([13 12  4 17 14 11  1]) = 0.5075656911260628
> seleccionada: J([13 12  4 17 14 11  2]) = 0.5194440421995228
> seleccionada: J([13 12  4 17 14 11  5]) = 0.5203864285203706
> seleccionada: J([13 12  4 17 14 11  7]) = 0.5221104956279339
> seleccionada: J([13 12  4 17 14 11 16]) = 0.5251885904846124
> seleccionada: J([13 12  4 17 14 11 1

 73%|███████▎  | 11/15 [00:00<00:00, 18.55it/s]

> seleccionada: J([13 12  4 17 14 11 38 24  0]) = 0.6026856447117938
> seleccionada: J([13 12  4 17 14 11 38 24  1]) = 0.6031008238593407
> seleccionada: J([13 12  4 17 14 11 38 24  2]) = 0.6415064603882038
> seleccionada: J([13 12  4 17 14 11 38 24  2  0]) = 0.6460787597165036
> seleccionada: J([13 12  4 17 14 11 38 24  2  1]) = 0.6880816598853435
> seleccionada: J([13 12  4 17 14 11 38 24  2  3]) = 0.7024411413431602
> seleccionada: J([13 12  4 17 14 11 38 24  2  3  0]) = 0.7045788145521438
> seleccionada: J([13 12  4 17 14 11 38 24  2  3  1]) = 0.7286722558507052
> seleccionada: J([13 12  4 17 14 11 38 24  2  3  5]) = 0.7311560563903592
> seleccionada: J([13 12  4 17 14 11 38 24  2  3 10]) = 0.7633611522457532
> seleccionada: J([13 12  4 17 14 11 38 24  2  3 10  0]) = 0.7668639884673083
> seleccionada: J([13 12  4 17 14 11 38 24  2  3 10  1]) = 0.8209288790928416
> seleccionada: J([13 12  4 17 14 11 38 24  2  3 10  8]) = 0.8411672119432343


100%|██████████| 15/15 [00:00<00:00, 18.83it/s]

> seleccionada: J([13 12  4 17 14 11 38 24  2  3 10  8  0]) = 0.8475453505881589
> seleccionada: J([13 12  4 17 14 11 38 24  2  3 10  8  1]) = 0.8636658405808332
> seleccionada: J([13 12  4 17 14 11 38 24  2  3 10  8 18]) = 0.8702867737959634
> seleccionada: J([13 12  4 17 14 11 38 24  2  3 10  8 20]) = 0.8996991119978857
> seleccionada: J([13 12  4 17 14 11 38 24  2  3 10  8 20  0]) = 0.9028733602680034
> seleccionada: J([13 12  4 17 14 11 38 24  2  3 10  8 20  1]) = 0.9153603748549644
> seleccionada: J([13 12  4 17 14 11 38 24  2  3 10  8 20  6]) = 0.9218440150292768
> seleccionada: J([13 12  4 17 14 11 38 24  2  3 10  8 20  7]) = 0.9224789173378138
> seleccionada: J([13 12  4 17 14 11 38 24  2  3 10  8 20 18]) = 0.9291302539572743
> seleccionada: J([13 12  4 17 14 11 38 24  2  3 10  8 20 28]) = 0.9327103713929925
> seleccionada: J([13 12  4 17 14 11 38 24  2  3 10  8 20 34]) = 0.9330453249563609
> seleccionada: J([13 12  4 17 14 11 38 24  2  3 10  8 20 34  0]) = 0.9376464943163947
>

In [ ]:
# Classification and Evaluation
from   sklearn.metrics   import confusion_matrix, accuracy_score
from   sklearn.neighbors import KNeighborsClassifier


In [ ]:
#División de datos
#from sklearn.model_selection import train_test_split

#X_entren, X_test, Y_entren , Y_test = train_test_split(X , Y, test_size=0.3)

In [ ]:
# Clasificación usando KNN con K vecinos
def Clasificador_KNN(Xtrain, ytrain, Xtest, K):
    # Normalización MinMax
    # Xtrain, a, b = minmax(Xtrain)
    # Xtest = Xtest * a + b

    # Entrenamiento
    knn = KNeighborsClassifier(n_neighbors=K)
    knn.fit(Xtrain, ytrain)
    print('Entrenando con ' + str(Xtrain.shape[0]) + ' muestras y probando con ' + str(Xtest.shape[0]) + ' muestras')

    # Prueba
    ypred = knn.predict(Xtest)
    return ypred

In [ ]:
def Precision_Matriz_Confusion(ytest, ypred):
    # Evaluación del desempeño
    acc = accuracy_score(ytest, ypred)
    print('Precisión en el conjunto de prueba = ' + str(acc * 100) + '%')
    C = confusion_matrix(ytest, ypred)
    print('Matriz de confusión:')
    print(C)
    return acc, C

In [ ]:
# EJEMPLO DE USO:
# Xtrain, ytrain, Xtest, ytest = DividirDatos(X, y, n)  # Las primeras n muestras de cada clase de X son entrenamiento, el resto es prueba
# ypred = Clasificador_KNN(Xtrain, ytrain, Xtest, 3)         # Cálculo de la predicción con un KNN de 3 vecinos
# acc, C = Precision_Matriz_Confusion(ytest, ypred)                # Precisión y matriz de confusión

In [ ]:
# Clasificación con datos originales (sin normalizar)
vecinos = 3  # número de vecinos para KNN
ypred = Clasificador_KNN(Xtrain_original, ytrain, Xtest_original, vecinos)  # Cálculo de la predicción con KNN de 3 vecinos
acc, C = Precision_Matriz_Confusion(ytest, ypred)  # Calcular precisión y matriz de confusión

Entrenando con 375 muestras y probando con 125 muestras
Precisión en el conjunto de prueba = 68.0%
Matriz de confusión:
[[16  3  1  1  4]
 [ 0 20  4  1  0]
 [ 1  3 16  1  4]
 [ 6  0  0 14  5]
 [ 0  5  0  1 19]]


In [ ]:
XX=Xtest_original
XX

array([[ 7.90000000e+01,  6.80000000e+01,  3.74500000e+03, ...,
         2.80762049e-01, -5.46705073e-01,  2.19873803e+00],
       [ 8.10000000e+01,  9.20000000e+01,  5.18600000e+03, ...,
         2.49513463e-01, -6.87610770e-01,  2.56556412e+00],
       [ 9.60000000e+01,  1.05000000e+02,  6.63400000e+03, ...,
         2.53066157e-01, -6.73044289e-01,  2.56655025e+00],
       ...,
       [ 9.00000000e+01,  8.60000000e+01,  5.21000000e+03, ...,
         3.10399795e-01, -9.52095287e-01,  3.16629139e+00],
       [ 9.90000000e+01,  1.02000000e+02,  9.46100000e+03, ...,
         9.81821795e-02,  1.01092010e-01,  1.91261600e+00],
       [ 9.90000000e+01,  1.02000000e+02,  9.46100000e+03, ...,
         9.81821795e-02,  1.01092010e-01,  1.91261600e+00]])

In [ ]:
# Encontrar las características constantes
constantes = np.all(XX == XX[0, :], axis=0)

# Imprimir las columnas constantes (True indica que la columna es constante)
print(constantes)

# Mostrar los índices de las características constantes
columnas_constantes = np.where(constantes)[0]
print("Características constantes:", columnas_constantes)

[False False False False False  True False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False]
Características constantes: [5]


In [ ]:
XXX=Xtrain_norm

col_con_nan = np.any(np.isnan(XXX), axis=0)

# Imprimir qué columnas tienen NaN
print("Columnas con NaN:", col_con_nan)

# Mostrar los índices de las columnas que tienen NaN
columnas_con_nan = np.where(col_con_nan)[0]
print("Índices de columnas con NaN:", columnas_con_nan)

Columnas con NaN: [False False False False False  True False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False]
Índices de columnas con NaN: [5]


In [ ]:
# Clasificación con datos normalizados del original
# Normalización MinMax
Xtrain_norm1, a, b = minmax(Xtrain_original)  # Calcular los parámetros de normalización (mínimo y máximo)
Xtest_norm = Xtest_original * a + b  # Aplicar la normalización a los datos de prueba

ypred = Clasificador_KNN(Xtrain_norm1, ytrain, Xtest_norm, vecinos)  # Cálculo de la predicción con KNN de 3 vecinos
acc, C = Precision_Matriz_Confusion(ytest, ypred)  # Calcular precisión y matriz de confusión

#ERROR PORQUE HAY CARACTERÍSTICAS CONSTANTES => DIVISIÓN POR CERO

ValueError: Input X contains NaN.
KNeighborsClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
#MINimax corregida
def minmax(X):
    ma = X.max(0)
    mi = X.min(0)
    md = ma - mi  # Rango de cada característica
    # Evitar división por cero reemplazando el rango 0 por 1
    md[md == 0] = 1
    a = 1 / md
    b = -mi / md
    return X * a + b, a, b

print(np.isnan(Xtrain_norm).sum(), np.isnan(Xtest_norm).sum())  # Esto te mostrará el número de valores NaN

# Normalización MinMax en los datos "limpios"
Xtrain_norm, a, b = minmax(Xtrain_original)  # Calcular los parámetros de normalización (mínimo y máximo)
Xtest_norm = Xtest_original * a + b  # Aplicar la normalización a los datos de prueba

# Verificar si hay valores NaN
print(np.isnan(Xtrain_norm).sum(), np.isnan(Xtest_norm).sum())  # Muestra el número de NaN en los datos

# Aplicar el clasificador KNN si no hay NaN
if np.isnan(Xtrain_norm).sum() == 0 and np.isnan(Xtest_norm).sum() == 0:
    ypred = Clasificador_KNN(Xtrain_norm, ytrain, Xtest_norm, vecinos)  # Cálculo de la predicción con KNN de 3 vecinos
    acc, C = Precision_Matriz_Confusion(ytest, ypred)  # Calcular precisión y matriz de confusión
else:
    print("Existen valores NaN en los datos, es necesario corregirlos.")


In [ ]:
# Clasificación con datos "limpios" (sin normalización)

ypred = Clasificador_KNN(Xtrain_clean, ytrain, Xtest_clean, n_neighbors)  # Cálculo de la predicción con KNN de 3 vecinos
acc, C = Precision_Matriz_Confusion(ytest, ypred)  # Calcular precisión y matriz de confusión

Entrenando con 375 muestras y probando con 125 muestras
Precisión en el conjunto de prueba = 68.0%
Matriz de confusión:
[[16  3  1  1  4]
 [ 0 20  4  1  0]
 [ 1  3 16  1  4]
 [ 6  0  0 14  5]
 [ 0  5  0  1 19]]


In [ ]:
# Normalización MinMax en los datos "limpios"
Xtrain_norm, a, b = minmax(Xtrain_clean)  # Calcular los parámetros de normalización en los datos limpios
Xtest_norm = Xtest_clean * a + b  # Aplicar la normalización a los datos de prueba limpios

In [ ]:
# Clasificación con datos limpios y normalizados

ypred = Clasificador_KNN(Xtrain_norm, ytrain, Xtest_norm, n_neighbors)  # Cálculo de la predicción con KNN de 3 vecinos
acc, C = Precision_Matriz_Confusion(ytest, ypred)  # Calcular precisión y matriz de confusión


Entrenando con 375 muestras y probando con 125 muestras
Precisión en el conjunto de prueba = 80.80000000000001%
Matriz de confusión:
[[17  0  2  6  0]
 [ 0 25  0  0  0]
 [ 1  0 24  0  0]
 [ 5  0  0 16  4]
 [ 0  0  0  6 19]]


In [ ]:
# Función SP100 para clasificación binaria
def sp100(caracteristicas, ypred):
    norm = (ypred.flatten() - ypred.min()).astype(bool)
    max_clase = norm.max()

    if max_clase > 1:
        raise Exception('sp100 solo funciona para dos clases')

    c1_caracteristicas = caracteristicas[norm == 1, :]

    min_carac = c1_caracteristicas.min(0)
    max_carac = c1_caracteristicas.max(0)

    z1 = (caracteristicas >= min_carac) & (caracteristicas <= max_carac)
    dr = z1.all(1)

    TP = (dr * norm).sum()
    FP = dr.sum() - TP

    TN = (~dr * ~norm).sum()
    return TN / (FP + TN)

# Función para calcular la puntuación de las características
def puntuacion(caracteristicas, ypred, *, metodo='fisher', param=None):
    if param is None:
        dn = ypred.max() - ypred.min() + 1  # Número de clases
        p = np.ones((dn, 1)) / dn
    else:
        p = param

    if metodo == 'mi':  # Información mutua
        raise NotImplementedError()

    # Relevancia máxima
    elif metodo == 'mr':
        raise NotImplementedError()

    # Redundancia mínima y relevancia máxima
    elif metodo == 'mrmr':
        raise NotImplementedError()

    # Fisher
    elif metodo == 'fisher':
        return jfisher(caracteristicas, ypred, p)

    elif metodo == 'sp100':
        return sp100(caracteristicas, ypred)

    else:
        return 0

# Selección secuencial de características (SFS)
def sfs(caracteristicas, ypred, n_caracteristicas, *, forzar=False, metodo='fisher', opciones=None, mostrar=False):

    N, M = caracteristicas.shape
    caracteristicas_restantes = set(np.arange(M))
    seleccionadas = list()
    caracteristicas_actuales = np.zeros((N, 0))
    if opciones is None:
        opciones = dict()

    def _calc_puntuacion(i):
        caracs = np.hstack([caracteristicas_actuales, caracteristicas[:, i].reshape(-1, 1)])
        return puntuacion(caracs, ypred, metodo=metodo, **opciones)

    if mostrar:
        _rango = tqdm.trange(
            n_caracteristicas, desc='Seleccionando características', unit_scale=True, unit=' características')
    else:
        _rango = range(n_caracteristicas)

    for _ in _rango:
        nueva_seleccion = max(caracteristicas_restantes, key=_calc_puntuacion)
        seleccionadas.append(nueva_seleccion)
        caracteristicas_restantes.remove(nueva_seleccion)
        caracteristicas_actuales = np.hstack(
            [caracteristicas_actuales, caracteristicas[:, nueva_seleccion].reshape(-1, 1)])

    return np.array(seleccionadas)

In [ ]:
# SFS implementation of Balu3
ss = sfs(Xtrain_norm,ytrain,p)
print(ss)

[13 12  4 17 14 11 38 24  2  3 10  8 20 34 28]


In [ ]:
Xtrain_sel = Xtrain_norm[:,ss]
Xtest_sel  = Xtest_norm[:,ss]

In [ ]:
ypred = Clasificador_KNN(Xtrain_sel,ytrain,Xtest_sel,n_neighbors)  # calculo de la prediccion con un KNN de 3 vecinos
acc,C = Precision_Matriz_Confusion(ytest,ypred)

Entrenando con 375 muestras y probando con 125 muestras
Precisión en el conjunto de prueba = 94.39999999999999%
Matriz de confusión:
[[24  0  1  0  0]
 [ 0 25  0  0  0]
 [ 0  0 25  0  0]
 [ 0  0  0 25  0]
 [ 0  0  0  6 19]]


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

K = 5   # número de clases
N = 100 # número de imágenes por clase

# Inicializar un DataFrame vacío con las columnas para almacenar todas las características
columns = [f'bg_{i}' for i in range(16)] + \
          [f'fd_{i}' for i in range(16)] + \
          [f'hu_{i}' for i in range(7)] + \
          [f'fl_{i}' for i in range(4)] + \
          [f'gu_{i}' for i in range(3)] + \
          ['class']  # Incluimos la columna para la etiqueta de clase

# Crear un DataFrame vacío con las columnas definidas
df = pd.DataFrame(columns=columns)

t = 0  # Inicializa un contador para llevar el índice total de imágenes procesadas

for j in range(K):  # Bucle sobre el número de clases
    for i in tqdm(range(N)):  # Bucle sobre las imágenes dentro de la clase, con barra de progreso
        # Lectura de la imagen
        img = imageload('LetrasABXYZ/char_', j+1, 2, i+1, 3, echo='off')

        # Dilatación de la imagen utilizando un disco de radio 3
        img3 = dilation(img, disk(3))  # Unir la imagen usando dilatación con un disco de radio 3

        # Binarización de la imagen: convertir la imagen en valores binarios (0 y 1)
        R = img3 > 0.5

        # Rellenar los agujeros en las regiones binarias y convertirlo a enteros (0 y 1)
        R = binary_fill_holes(R).astype(int)

        # Extracción de características:
        fd = fourierdes(R)  # Extraer descriptores de Fourier
        hu = hugeo(R)       # Extraer momentos de Hu
        fl = flusser(R)     # Extraer momentos de Flusser
        gu = gupta(R)       # Extraer momentos de Gupta
        bg = basicgeo(R)    # Extraer características geométricas básicas

        # Crear un diccionario con todas las características extraídas y la clase
        features = {
            **{f'bg_{k}': bg[k] for k in range(16)},
            **{f'fd_{k}': fd[k] for k in range(16)},
            **{f'hu_{k}': hu[k] for k in range(7)},
            **{f'fl_{k}': fl[k] for k in range(4)},
            **{f'gu_{k}': gu[k] for k in range(3)},
            'class': j
        }

        # Añadir las características como una nueva fila al DataFrame
        df = df.append(features, ignore_index=True)

        # Incrementar el contador de imágenes procesadas
        t += 1

# Mostrar el DataFrame resultante
import ace_tools as tools; tools.display_dataframe_to_user(name="Caracteristicas de imágenes", dataframe=df)
